<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>COMP 521 Files and Databases Fall 2020</h1>
<h1 style="font-size: 200%;">Problem Set 3</h1>
</div>

---
In the following questions you will need to write Python3 programs to perform queries on and modify the <a href="http://csbio.unc.edu/mcmillan/Media/NCCOVID19.db" download>NCCOVID19.db</a> database used in Problem Set 2. **You are only allowed to import the standard "sqlite3" and the "time" Python packages.**

In [ ]:
import iSQL
query = iSQL.parser("NCCOVID19.db")

---
**Problem 1:** The file <a href="http://csbio.unc.edu/mcmillan/Media/NC2018Mortality.csv" download="NC2018Mortality.csv">NC2018Mortality.csv</a> contains a list of deaths by race and sex in each of North Carolina's counties. Write a Python3 program that creates a new table in the NCCOVID19.db database and then loads that table into the database using values from this file. The table should have the following schema:

<table style="border-collapse: collapse; border: 1px solid black;"><tbody>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <th colspan="7" style="text-align:center; background-color: skyblue;">Mortality</th></tr>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <td style="border-collapse: collapse; border: 1px solid black;"><u>fips</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>year</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>race</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>sex</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;">count</td>
</tr>
</tbody></table>

Do not parse the file using pandas or Python's built-in "csv" modules "reader" method. This will require that you open the file, read its contents, split them into lines, and extract the header line.

In [1]:
import sqlite3

db = sqlite3.connect('NCCOVID19.db')
cursor = db.cursor()

fipsnameslist = cursor.execute("""SELECT C.name, C.fips FROM County C""").fetchall()
fipslist = {}
for elt in fipsnameslist:
    fipslist[elt[0].lower()] = elt[1]

cursor.execute("""DROP TABLE IF EXISTS Mortality""")
cursor.execute("""CREATE TABLE Mortality (
                      fips    INTEGER,
                      year    INTEGER,
                      race    VARCHAR,
                      sex     VARCHAR,
                      count   INTEGER,
                      PRIMARY KEY(fips, year, race, sex),
                      FOREIGN KEY(fips) REFERENCES Demographics(fips),
                      FOREIGN KEY(year) REFERENCES Demographics(year),
                      FOREIGN KEY(race) REFERENCES Demographics(race),
                      FOREIGN KEY(sex)  REFERENCES Demographics(sex))""")

with open('NC2018Mortality.csv', 'r') as csvfile:
    mort = [row.rstrip().split(',') for row in csvfile]

ret = []
for idx in range(1, len(mort)):    
    ret.append((fipslist[mort[idx][0].lower()], 'white', 'male', int(mort[idx][1])))
    ret.append((fipslist[mort[idx][0].lower()], 'white', 'female', int(mort[idx][2])))
    ret.append((fipslist[mort[idx][0].lower()], 'black', 'male', int(mort[idx][3])))
    ret.append((fipslist[mort[idx][0].lower()], 'black', 'female', int(mort[idx][4])))
    ret.append((fipslist[mort[idx][0].lower()], 'asian', 'male', int(mort[idx][5])))
    ret.append((fipslist[mort[idx][0].lower()], 'asian', 'female', int(mort[idx][6])))
    ret.append((fipslist[mort[idx][0].lower()], 'aian', 'male', int(mort[idx][7])))
    ret.append((fipslist[mort[idx][0].lower()], 'aian', 'female', int(mort[idx][8])))
    ret.append((fipslist[mort[idx][0].lower()], 'other', 'male', int(mort[idx][9])))
    ret.append((fipslist[mort[idx][0].lower()], 'other', 'female', int(mort[idx][10])))

for idx in range(len(ret)):
    cursor.execute("""INSERT INTO Mortality(fips, year, race, sex, count) VALUES(?, 2018, ?, ?, ?)""",
                  (ret[idx][0], ret[idx][1], ret[idx][2], ret[idx][3]))

db.commit()
db.close()

---
**Problem 2:** Use the new table that you created in **Problem 1** to create the following Python3 function that estimates the number of 2018 deaths in each demographic category for a given county. These estimates are based on the assumption that *the number of mortality deaths in each demographic's age range is proportional to the 2018 population fraction in that same demogrphic's age range*. Your function should return a list of <code>(race,sex,agelo,agehi,count)</code> tuples, where the sum of all counts should match the total number of 2018 deaths in that county. All <code>counts</code> should be integers. All of your code must be within the body of the given function.

In [2]:
import sqlite3

def covidDemographicsEstimate(countyName):
    demographicGroup = []
    db = sqlite3.connect("NCCOVID19.db")
    #db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()

    groupDemographic = []
    groupDemographic = cursor.execute(f"""SELECT A.race, A.sex, A.agelo, A.agehi, CAST((A.count) AS INT) AS casted,
                                                (A.count - CAST((A.count) AS INT)) AS remainder
                                          FROM (SELECT T.fips, T.race, T.sex, T.agelo, T.agehi, T.ratio*MM.count AS count
                                                FROM (SELECT D.fips, D.race, D.sex, D.agelo, D.agehi,
                                                            (1.0*D.count)/Total.count AS ratio
                                                      FROM Demographics D, (SELECT D.fips, D.race, D.sex,
                                                                                   SUM(D.count) AS count
                                                                            FROM Demographics D
                                                                            WHERE D.year=2018
                                                                            GROUP BY D.fips, D.race, D.sex) AS Total
                                                      WHERE D.year=2018 AND D.fips=Total.fips AND D.race=Total.race
                                                                                              AND D.sex=Total.sex) AS T
                                                LEFT JOIN (SELECT M.fips, M.race, M.sex, M.count
                                                           FROM Mortality M) AS MM
                                                ON MM.fips=T.fips AND MM.race=T.race AND MM.sex=T.sex) AS A, County C
                                          WHERE A.fips=C.fips AND C.name='{countyName}'
                                          ORDER BY remainder DESC""").fetchall()
    totalRem = 0
    for idx in range(len(groupDemographic)):
        totalRem += groupDemographic[idx][5]
    totalRem = round(totalRem)
    
    for idx in range(totalRem):
        demographicGroup.append((groupDemographic[idx][0], groupDemographic[idx][1],
                                 groupDemographic[idx][2], groupDemographic[idx][3],
                                 groupDemographic[idx][4] + 1))
    
    for idx in range(totalRem, len(groupDemographic)):
        demographicGroup.append((groupDemographic[idx][0], groupDemographic[idx][1],
                                 groupDemographic[idx][2], groupDemographic[idx][3],
                                 groupDemographic[idx][4]))
    
    #total = 0
    #for elt in demographicGroup:
    #    total += elt[4]
    #print(total)

    return demographicGroup
covidDemographicsEstimate('Alamance')

[('black', 'female', 85, 99, 3),
 ('black', 'female', 0, 2, 6),
 ('white', 'male', 15, 15, 9),
 ('black', 'male', 6, 9, 10),
 ('white', 'female', 85, 99, 24),
 ('black', 'female', 55, 59, 12),
 ('black', 'male', 20, 24, 12),
 ('black', 'male', 3, 4, 5),
 ('white', 'female', 35, 44, 80),
 ('other', 'female', 25, 34, 1),
 ('white', 'female', 65, 74, 75),
 ('black', 'female', 18, 19, 4),
 ('black', 'female', 15, 15, 2),
 ('white', 'female', 75, 84, 45),
 ('other', 'female', 10, 13, 1),
 ('black', 'female', 10, 13, 8),
 ('black', 'female', 5, 5, 2),
 ('black', 'male', 55, 59, 12),
 ('white', 'male', 3, 4, 14),
 ('white', 'male', 85, 99, 11),
 ('other', 'male', 10, 13, 1),
 ('other', 'male', 6, 9, 1),
 ('other', 'male', 25, 34, 1),
 ('white', 'male', 65, 74, 62),
 ('black', 'male', 25, 34, 23),
 ('white', 'female', 18, 19, 25),
 ('white', 'female', 20, 24, 53),
 ('other', 'female', 6, 9, 1),
 ('white', 'male', 75, 84, 33),
 ('white', 'male', 10, 13, 33),
 ('asian', 'female', 65, 74, 1),
 ('

---
**Problem 3:** In the space below write a SQL query that generates a list of (countyName, covidDeaths, totalDeaths2018, percentage) for all counties for which the 2020 Covid19 deaths exceed 5% of the total deaths for that same county in 2018.

---
**Problem 4:** In the space below write a Python3 function that returns a list of all dates for which the given "fips" number accounts for at least the given "percent" of the total Covid19 cases on that date. As an example, the NCCOVID19.db database includes Covid19 records that are not associated with any county. These records have a fips of 0. When your function is called with <code>AtLeastPercent(0, 10.0)</code> it should return the following list of dates:

     ['2020-06-05', '2020-06-10', '2020-06-15', '2020-08-12']

In [3]:
import sqlite3

def AtLeastPercent(fips, percent):
    dateList = []
    db = sqlite3.connect("NCCOVID19.db")
    #db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()
    
    dateList = cursor.execute(f"""SELECT date
                                  FROM (SELECT C19.fips, C19.date, Tmp.cases AS part, SUM(C19.cases) AS total
                                        FROM Covid19 C19, (SELECT C19.fips, C19.date, C19.cases
                                                           FROM Covid19 C19
                                                           WHERE C19.cases>0 AND C19.fips={fips}) AS Tmp
                                        WHERE C19.date=Tmp.date
                                        GROUP BY C19.date)
                                  WHERE 100.0*part/total>={percent}
                               """).fetchall()
    
    for idx in range(len(dateList)):
        tmp = dateList[idx][0]
        dateList[idx] = tmp
    
    return dateList
AtLeastPercent(0, 10.0)

['2020-06-05', '2020-06-10', '2020-06-15', '2020-08-12']

---
**Problem 5:** The following query was designed to identify dates where two Counties reported the same number of cases on the same date:<br>
        <pre>
        SELECT C1.name, C2.name, V1.date, V1.cases
        FROM County C1, Covid19 V1, County C2, Covid19 V2
        WHERE C1.fips = V1.fips AND C2.fips = V2.fips AND
              V1.cases = V2.cases AND V1.date = V2.date AND
              V1.fips < V2.fips AND V1.cases >= 1
        </pre>
        
This query is slow using the default NCCOVID19.db. Write a Python3 program to execute and time this query along with time to save all rows that it returns into a list for subsequent processing. You should verify that your list actually extracts data from the rows. In the cell provided below give the number of rows returned and the time required (no code please).

In [4]:
# Without for-loop
import sqlite3
import time

db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
db.text_factory = str
cursor = db.cursor()

begin = time.time()
    
query = cursor.execute("""SELECT C1.name, C2.name, V1.date, V1.cases
                          FROM County C1, Covid19 V1, County C2, Covid19 V2
                          WHERE C1.fips=V1.fips AND C2.fips=V2.fips AND
                                V1.cases=V2.cases AND V1.date=V2.date AND
                                V1.fips<V2.fips AND V1.cases>=1""").fetchall()

avg = 0
for elt in query:
    avg += elt[3]
print(avg / len(query))
    
end = time.time()
elapsedTime = end - begin
    
count = len(query)

print(elapsedTime)
print(count)

3.522627301871319
0.2556426525115967
26826


In [5]:
# Using for-loop (50 iterations)
import sqlite3
import time

db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
db.text_factory = str
cursor = db.cursor()

averageTime = 0;

for k in range(50):
    begin = time.time()
    
    query = cursor.execute("""SELECT C1.name, C2.name, V1.date, V1.cases
                              FROM County C1, Covid19 V1, County C2, Covid19 V2
                              WHERE C1.fips=V1.fips AND C2.fips=V2.fips AND
                                    V1.cases=V2.cases AND V1.date=V2.date AND
                                    V1.fips<V2.fips AND V1.cases>=1""").fetchall()

    end = time.time()
    averageTime += (end - begin)
    
elapsedTime = averageTime / 50
count = len(query)

print(elapsedTime)
print(count)

0.25906471729278563
26826


---
**Problem 6:** Next create an index to improve the performance of this query. In the cell provided below give
the "CREATE INDEX" statement that you found provides the best performance improvement. Also include
the time required for the query after creating the index you chose.

In [6]:
import sqlite3
import time

db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
db.text_factory = str
cursor = db.cursor()

cursor.execute("CREATE INDEX IF NOT EXISTS idx ON Covid19(cases, date, fips)")
begin = time.time()
    
query = cursor.execute("""SELECT C1.name, C2.name, V1.date, V1.cases
                          FROM County C1, Covid19 V1, County C2, Covid19 V2
                          WHERE C1.fips=V1.fips AND C2.fips=V2.fips AND
                                V1.cases=V2.cases AND V1.date=V2.date AND
                                V1.fips<V2.fips AND V1.cases>=1""").fetchall()

avg = 0
for elt in query:
    avg += elt[3]
print(avg / len(query))

end = time.time()
elapsedTime = end - begin

cursor.execute("""DROP INDEX idx""").fetchall()

print(elapsedTime)

0.18890047073364258
